## General prerprocess data for baseline models.

## Preprocess for both SARIMA and HWES, we need fixed time interval:

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from gluonts.dataset.common import ListDataset
import json
import os
import math

C:\Users\davyx\anaconda3\envs\satcast\Lib\site-packages\gluonts\json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [1]:
df = pd.read_parquet('../data/processed/cleaned.parquet')
price_cols_to_drop = [col for col in df.columns if col.startswith("price_") and col != "price_USD"]
df.drop(columns=price_cols_to_drop, inplace=True) #Delete all price rows except usd
df.head()

NameError: name 'pd' is not defined

Check if all the time interval are fixed.

In [3]:
time_diffs = df.index.to_series().diff()
print("Unique time intervals:")
print(time_diffs.value_counts())

most_common_interval = time_diffs.mode()[0]
print(f"\nMost common interval: {most_common_interval}")


Unique time intervals:
timestamp
0 days 00:05:00    4291
0 days 00:05:01    3555
0 days 00:04:59    3555
0 days 00:05:04     128
0 days 00:04:56     127
0 days 00:05:03     110
0 days 00:04:57     110
0 days 00:04:55       3
0 days 00:05:05       2
0 days 00:05:13       1
0 days 00:02:43       1
0 days 00:07:17       1
0 days 00:07:03       1
0 days 00:04:44       1
0 days 00:05:57       1
0 days 00:02:57       1
0 days 00:04:07       1
0 days 00:04:58       1
0 days 00:07:00       1
0 days 00:03:03       1
0 days 00:08:38       1
0 days 00:01:21       1
0 days 00:08:46       1
0 days 00:01:15       1
0 days 00:06:29       1
0 days 00:03:32       1
0 days 00:06:23       1
0 days 00:03:36       1
Name: count, dtype: int64

Most common interval: 0 days 00:05:00


In [4]:
# Resample to 15-minute regular intervals, averaging or interpolating values
# No actual missing data, just values not aligned exactly on the desired resampling grid, so we can interpolate the data.
df_resampled = df.resample("15min").mean()
df_resampled_hwes = df.resample("5min").mean()
# Linear interpolation is safe here due to small timing variations
df_resampled = df_resampled.interpolate(method='linear')
df_resampled_hwes = df_resampled_hwes.interpolate(method='linear')

In [5]:
#SARIMA need stationarity check
ts = df_resampled['recommended_fee_fastestFee']
adf_result = adfuller(ts.dropna())
print(f"ADF Statistic: {adf_result[0]}")
print(f"p-value: {adf_result[1]}")


ADF Statistic: -11.156553451590447
p-value: 2.856371654220604e-20


The p-value is much less than 0.05, which strongly indicates the series is stationary.
This means: no differencing is needed before fitting ARIMA or SARIMA.

In [6]:
df_resampled.to_parquet("../data/processed/preprocessed_sarima.parquet")
df_resampled_hwes.to_parquet("../data/processed/preprocessed_hwes.parquet")

In [7]:
df_resampled.to_parquet("../data/processed/preprocessed_prophet.parquet")

## Preprocess for XGboost

In [8]:
#We will need lagging feature for XGboost. Since each hour has 4 data points (60 ÷ 15), 48 hours = 192 lag steps.
def create_lag_features_fast(df, target_col, lags):
    lagged_dfs = [
        df[[target_col]].shift(lag).rename(columns={target_col: f'{target_col}_lag_{lag}'})
        for lag in lags
    ]
    return pd.concat([df] + lagged_dfs, axis=1)
lags = range(1, 193)  # 48 hours of 15-minute intervals
df_xgboost = create_lag_features_fast(df_resampled, 'recommended_fee_fastestFee', lags)
df_xgboost.head()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,recommended_fee_fastestFee_lag_183,recommended_fee_fastestFee_lag_184,recommended_fee_fastestFee_lag_185,recommended_fee_fastestFee_lag_186,recommended_fee_fastestFee_lag_187,recommended_fee_fastestFee_lag_188,recommended_fee_fastestFee_lag_189,recommended_fee_fastestFee_lag_190,recommended_fee_fastestFee_lag_191,recommended_fee_fastestFee_lag_192
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:00:00,2.131827e+06,997874.250000,1030.000000,2.077770e+06,1.300139,2.000000,2.000000,2.000000,2.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:15:00,1.801526e+06,997925.916667,2627.000000,3.407913e+06,2.473960,3.000000,2.666667,2.666667,2.666667,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:30:00,2.055350e+06,997965.916667,922.000000,1.855358e+06,1.355898,2.000000,2.000000,2.000000,2.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:45:00,2.006800e+06,997970.083333,1296.666667,2.167251e+06,1.536092,2.333333,2.333333,2.333333,2.333333,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 03:00:00,1.773658e+06,997960.333333,2725.333333,3.462741e+06,2.467659,3.333333,3.000000,3.000000,3.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_xgboost.to_parquet("../data/processed/preprocessed_xgboost.parquet")

## Advanced-deepar

In [10]:
df.index = pd.to_datetime(df.index) 
df_deepar = df.copy()
df_deepar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11901 entries, 2025-03-05 02:14:41 to 2025-04-15 09:54:42
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   mempool_blocks_blockSize                  11900 non-null  float64
 1   mempool_blocks_blockVSize                 11900 non-null  float64
 2   mempool_blocks_nTx                        11900 non-null  float64
 3   mempool_blocks_totalFees                  11900 non-null  float64
 4   mempool_blocks_medianFee                  11900 non-null  float64
 5   recommended_fee_fastestFee                11901 non-null  float64
 6   recommended_fee_halfHourFee               11901 non-null  float64
 7   recommended_fee_hourFee                   11901 non-null  float64
 8   recommended_fee_economyFee                11901 non-null  float64
 9   recommended_fee_minimumFee                11901 non-null  float64
 10 

In [11]:
# Step 1: Extract timestamp and target
start_timestamp = df_deepar.index[0]  # assumes index is datetime
target_values = df_deepar["recommended_fee_fastestFee"].values.tolist()
start_timestamp

Timestamp('2025-03-05 02:14:41')

In [12]:
# Step 2: add dow, tod

# 1. Day of week (one-hot)
df_deepar['day_of_week'] = df_deepar.index.dayofweek
day_of_week_ohe = pd.get_dummies(df_deepar['day_of_week'], prefix='dow')

# 2. Time of day (cyclical features)
df_deepar['hour'] = df_deepar.index.hour
df_deepar['minute'] = df_deepar.index.minute
df_deepar['minute_of_day'] = df_deepar['hour'] * 60 + df_deepar['minute']
df_deepar['tod_sin'] = np.sin(2 * np.pi * df_deepar['minute_of_day'] / 1440)
df_deepar['tod_cos'] = np.cos(2 * np.pi * df_deepar['minute_of_day'] / 1440)

# 3. Merge all
df_deepar = pd.concat([df_deepar, day_of_week_ohe], axis=1)

# Preview
df_deepar.head()


,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,minute_of_day,tod_sin,tod_cos,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:14:41,2131827.0,997874.25,1030.0,2077770.0,1.300139,2.0,2.0,2.0,2.0,2.0,...,134,0.551937,0.833886,False,False,True,False,False,False,False
2025-03-05 02:19:42,1986259.0,997882.00,1915.0,2663152.0,2.000000,2.0,2.0,2.0,2.0,2.0,...,139,0.569997,0.821647,False,False,True,False,False,False,False
2025-03-05 02:24:41,1814202.0,997958.25,2710.0,3423846.0,2.413300,3.0,3.0,3.0,3.0,2.0,...,144,0.587785,0.809017,False,False,True,False,False,False,False
2025-03-05 02:29:42,1604116.0,997937.50,3256.0,4136741.0,3.008581,4.0,3.0,3.0,3.0,2.0,...,149,0.605294,0.796002,False,False,True,False,False,False,False
2025-03-05 02:34:42,2200654.0,997987.75,1015.0,1890732.0,1.362632,2.0,2.0,2.0,2.0,2.0,...,154,0.622515,0.782608,False,False,True,False,False,False,False


In [13]:
#Step3: change all fee from wide to long
fee_columns = [
    'recommended_fee_fastestFee',
    'recommended_fee_halfHourFee',
    'recommended_fee_hourFee',
    'recommended_fee_economyFee',
    'recommended_fee_minimumFee'
]
df_long = df_deepar[fee_columns].copy()
df_long["timestamp"] = df.index  # restore timestamp if it's the index

# Melt to long format
df_long = df_long.melt(
    id_vars=["timestamp"],
    value_vars=fee_columns,
    var_name="fee_type",
    value_name="fee_value"
)
df_merged = pd.merge(df_long, df_deepar, on="timestamp", how="left")
df_merged_sorted = df_merged.sort_values(by=["fee_type", "timestamp"]).reset_index(drop=True)
df_merged_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59505 entries, 0 to 59504
Data columns (total 77 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   timestamp                                 59505 non-null  datetime64[ns]
 1   fee_type                                  59505 non-null  object        
 2   fee_value                                 59505 non-null  float64       
 3   mempool_blocks_blockSize                  59500 non-null  float64       
 4   mempool_blocks_blockVSize                 59500 non-null  float64       
 5   mempool_blocks_nTx                        59500 non-null  float64       
 6   mempool_blocks_totalFees                  59500 non-null  float64       
 7   mempool_blocks_medianFee                  59500 non-null  float64       
 8   recommended_fee_fastestFee                59505 non-null  float64       
 9   recommended_fee_halfHourFee 

In [14]:
#Step 4: Create list dataset for deepar
print(type(df_merged_sorted["timestamp"].iloc[0])) 
fee_type_to_id = {ft: i for i, ft in enumerate(df_merged_sorted["fee_type"].unique())}
external_cols = df_merged_sorted.columns.difference(["timestamp", "fee_type", "fee_value"])

series_list = []

for fee_type, group in df_merged_sorted.groupby("fee_type"):
    group = group.sort_values("timestamp")

    series = {
        "start": group["timestamp"].iloc[0].strftime("%Y-%m-%d %H:%M:%S"),
        "target": group["fee_value"].tolist(),
        "feat_dynamic_real": group[external_cols].astype(float).T.values.tolist(),
        "feat_static_cat": [int(fee_type_to_id[fee_type])]
    }

    series_list.append(series)



<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [15]:
feat_dynamic_real_names = list(external_cols)

def sanitize_for_json(obj):
    """Recursively replace NaN with None so JSON can parse it."""
    if isinstance(obj, float) and math.isnan(obj):
        return None
    elif isinstance(obj, list):
        return [sanitize_for_json(x) for x in obj]
    elif isinstance(obj, dict):
        return {k: sanitize_for_json(v) for k, v in obj.items()}
    else:
        return obj
with open("../data/processed/deepar_dataset.jsonl", "w") as f:
    for item in series_list:
        clean_item = sanitize_for_json(item)
        json.dump(clean_item, f)
        f.write("\n")

# Save mapping
with open("../data/processed/fee_type_to_id.json", "w") as f:
    json.dump(fee_type_to_id, f)

with open("../data/processed/feat_dynamic_real_names.json", "w") as f:
    json.dump(feat_dynamic_real_names, f, indent=2)


## TFT

In [19]:
df_tft = df_resampled.copy()
df_tft.info()
df_tft.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3968 entries, 2025-03-05 02:00:00 to 2025-04-15 09:45:00
Freq: 15min
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   mempool_blocks_blockSize                  3968 non-null   float64
 1   mempool_blocks_blockVSize                 3968 non-null   float64
 2   mempool_blocks_nTx                        3968 non-null   float64
 3   mempool_blocks_totalFees                  3968 non-null   float64
 4   mempool_blocks_medianFee                  3968 non-null   float64
 5   recommended_fee_fastestFee                3968 non-null   float64
 6   recommended_fee_halfHourFee               3968 non-null   float64
 7   recommended_fee_hourFee                   3968 non-null   float64
 8   recommended_fee_economyFee                3968 non-null   float64
 9   recommended_fee_minimumFee                3968 non-null   f

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,mempool_fee_histogram_bin_70_75,mempool_fee_histogram_bin_75_80,mempool_fee_histogram_bin_80_85,mempool_fee_histogram_bin_85_90,mempool_fee_histogram_bin_90_95,mempool_fee_histogram_bin_95_100,mempool_fee_histogram_bin_100_150,mempool_fee_histogram_bin_150_200,mempool_fee_histogram_bin_200_250,mempool_fee_histogram_bin_250_300
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:00:00,2.131827e+06,997874.250000,1030.000000,2.077770e+06,1.300139,2.000000,2.000000,2.000000,2.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-05 02:15:00,1.801526e+06,997925.916667,2627.000000,3.407913e+06,2.473960,3.000000,2.666667,2.666667,2.666667,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-05 02:30:00,2.055350e+06,997965.916667,922.000000,1.855358e+06,1.355898,2.000000,2.000000,2.000000,2.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-05 02:45:00,2.006800e+06,997970.083333,1296.666667,2.167251e+06,1.536092,2.333333,2.333333,2.333333,2.333333,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-05 03:00:00,1.773658e+06,997960.333333,2725.333333,3.462741e+06,2.467659,3.333333,3.000000,3.000000,3.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
fee_cols = [
    "recommended_fee_fastestFee",
    "recommended_fee_hourFee",
    "recommended_fee_halfHourFee",
    "recommended_fee_economyFee",
    "recommended_fee_minimumFee"
]

df_long = df_tft[fee_cols].copy()
df_long["timestamp"] = df_tft.index
df_long = df_long.melt(id_vars=["timestamp"], var_name="series_id", value_name="target")
features = df_tft.drop(columns=fee_cols)
features = features.reset_index()
df_long = df_long.merge(features, on="timestamp", how="left")
df_long = df_long.sort_values(["series_id", "timestamp"]).reset_index(drop=True)
df_long["time_idx"] = df_long.groupby("series_id").cumcount()

df_long["hour"] = df_long["timestamp"].dt.hour
df_long["minute"] = df_long["timestamp"].dt.minute
df_long["day_of_week"] = df_long["timestamp"].dt.dayofweek
df_long["month"] = df_long["timestamp"].dt.month
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19840 entries, 0 to 19839
Data columns (total 64 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   timestamp                                 19840 non-null  datetime64[ns]
 1   series_id                                 19840 non-null  object        
 2   target                                    19840 non-null  float64       
 3   mempool_blocks_blockSize                  19840 non-null  float64       
 4   mempool_blocks_blockVSize                 19840 non-null  float64       
 5   mempool_blocks_nTx                        19840 non-null  float64       
 6   mempool_blocks_totalFees                  19840 non-null  float64       
 7   mempool_blocks_medianFee                  19840 non-null  float64       
 8   mempool_count                             19840 non-null  float64       
 9   mempool_vsize               

In [18]:
df_long.to_parquet("../data/processed/processed_tft.parquet", index=False)
